In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import json
import time
import numpy as np
import pandas as pd
import pickle
import os

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

import warnings
warnings.filterwarnings("ignore")

In [3]:
with open('./config.json', 'r') as fo:
    conf = json.loads(fo.read())
kib_login = conf['kib_login']
kib_password = conf['kib_password']
data_folder = './parsed_data'

# Пробуем парсинг К&Б через selenium

In [4]:
from src.kib_scrapper import LocalCatalogItems, KibCatalog, ShopSelector

In [5]:
driver = webdriver.Safari()

In [6]:
catalog = KibCatalog(driver, kib_login=kib_login, kib_password=kib_password, save_path=data_folder)

In [7]:
cities = []
for macro, d in catalog.shop_list.items():
    cities += list(d.keys())

cities_need = """КАЗАНЬ
САМАРА
ЯРОСЛАВЛЬ
НИЖНИЙ НОВГОРОД
МОСКВА
КРАСНОДАР
САНКТ-ПЕТЕРБУРГ
НОВОСИБИРСК
Екатеринбург
Пермь
ВОРОНЕЖ
ТУЛА
Ростов-на-Дону
Челябинск"""
cities_need = [c.lower() for c in cities_need.split('\n')]
regions_to_parse = []
for city in cities_need:
    for macro, d in catalog.shop_list.items():
        found = False
        for c in d.keys():
            if city in c.lower():
                found = True
                regions_to_parse.append((macro, c))
                break
        if found:
            break
    else:
        print(f'Не получилось найти город {city}')

adresses = []
for macro, city in regions_to_parse:
    adresses.append((macro, city, np.random.choice(catalog.shop_list[macro][city])))

In [7]:
adresses = [
    ('Республика Татарстан', 'Казань', 'ул. Маршала Чуйкова, 63а'),
    ('Самарская область', 'Самара', 'ул. Советской Армии, 17'),
    ('Ярославская область', 'Ярославль', 'пр-кт Ленинградский, 66а'),
    ('Нижегородская область', 'Нижний Новгород', 'ул. Василия Иванова, 9'),
    ('Москва', 'Москва', 'ул. Весенняя, 4'),
    ('Краснодарский край', 'Краснодар', 'ул. Агрохимическая, 117/1'),
    ('Санкт-Петербург', 'Санкт-Петербург', 'пр-т Пискаревский, 10а'),
    ('Новосибирская область', 'Новосибирск', 'ул. Комсомольская, 2/2'),
    ('Свердловская область', 'Екатеринбург', 'ул. Первомайская, 35'),
    ('Пермский край', 'Пермь', 'ул. Уинская, 35'),
    ('Воронежская область', 'Воронеж', 'ул. Березовая роща, 62б'),
    ('Тульская область', 'Тула', 'ул. Кутузова, 43'),
    ('Ростовская область', 'Ростов-на-Дону', 'б-р Комарова, 20'),
    ('Челябинская область', 'Челябинск', 'ул. Доватора, 42')
]

**Парсинг всего каталога**

In [9]:
# %%time
# # catalog = KibCatalog(driver, ['Москва', 'Курская область'], ['Вода, соки, напитки'])
# catalog = KibCatalog(driver, kib_login=kib_login, kib_password=kib_password, save_path=data_folder, regions = )
# # with open('all_kib_catalog.pickle', 'wb') as fo:
# #     pickle.dump(catalog.catalogs, fo)
# # catalog.parse_all_catalogs(['Бакалея, чай, кофе'])
# for i in range(10):
#     try:
#         catalog.parse_all_catalogs()
#     except:
#         driver.quit()
#         driver = webdriver.Safari()
#         catalog = KibCatalog(driver)

**Парсинг отдельных магазинов**

In [8]:
'kib_Свердловская область_Екатеринбург_ул. Первомайская, 35.csv' in os.listdir('./parsed_data')

False

In [11]:
'kib_Свердловская область_Екатеринбург_ул. Первомайская, 35.csv' in list(os.listdir('./parsed_data'))

False

In [10]:
os.listdir('./parsed_data')

['kib_Республика Татарстан_Казань_ул. Маршала Чуйкова, 63а.csv',
 '.DS_Store',
 'kib_Самарская область_Самара_ул. Советской Армии, 17.csv',
 'kib_Краснодарский край_Краснодар_ул. Агрохимическая, 117-1.csv',
 'kib_Пермский край_Пермь_ул. Уинская, 35.csv',
 'kib_Санкт-Петербург_Санкт-Петербург_пр-т Пискаревский, 10а.csv',
 'shops_info_old.pickle',
 'kib_Нижегородская область_Нижний Новгород_ул. Василия Иванова, 9.csv',
 'kib_Свердловская область_Екатеринбург_ул. Первомайская, 35.csv',
 'shops_info.pickle',
 'kib_Ярославская область_Ярославль_пр-кт Ленинградский, 66а.csv',
 'kib_Москва_Москва_ул. Весенняя, 4.csv',
 '.ipynb_checkpoints',
 'kib_parsing_2022_01_30.csv',
 'kib_Новосибирская область_Новосибирск_ул. Комсомольская, 2-2.csv']

In [12]:
# adresses = [
# #     ('Москва', 'Москва', 'пер. Лялин, 22'),
# #     ('Москва', 'Москва', 'б-р Академика Ландау, 3'),
#     ('Москва', 'Москва', 'Михайловский Верхн. 2-й проезд, 6')
# ]
for ad in adresses[10:]:
    tag = '_'.join(ad).replace('/', '-')
    filename = 'kib_' + tag + '.csv'
    print(filename)
    if filename in os.listdir('./parsed_data'):
        print('already done')
        continue
    
    file_path = './parsed_data/' + filename
    items = catalog.parse_catalog(*ad)
    df = items.get_dataframe()
    
    df.to_csv(file_path, index=False)
    driver.get('https://krasnoeibeloe.ru')
    time.sleep(10)

kib_Воронежская область_Воронеж_ул. Березовая роща, 62б.csv
parsing Вино импорт
parsing Вино Россия
parsing Вино с оценкой
parsing Вино игристое, вермут 
parsing Водка, настойки
parsing Виски, бурбон
parsing Коньяк, арманьяк
parsing Текила, ром, ликер
parsing Вода, соки, напитки
parsing Мясо, рыба, икра
parsing Молоко, сыры, яйцо
parsing Бакалея, чай, кофе
parsing Консервация, салаты
parsing Фрукты, овощи
Не удалось загрузить каталог
parsing Шоколад, конфеты, мороженое
parsing Хлеб, выпечка, печенье
parsing Снэки
parsing Пиво импорт
parsing Пиво Россия
parsing Коктейли
Не удалось загрузить каталог
parsing Электронные одноразовые сигареты
Не удалось загрузить каталог
parsing Logic, IQOS, Juul, Ploom, Glo
Не удалось загрузить каталог
parsing Для кальяна
Не удалось загрузить каталог
parsing Промтовары, игрушки
parsing Корм для животных
kib_Тульская область_Тула_ул. Кутузова, 43.csv
parsing Вино импорт
parsing Вино Россия
parsing Вино с оценкой
parsing Вино игристое, вермут 
parsing Водка,

In [4]:
files = os.listdir('./parsed_data')
df = pd.read_csv('./parsed_data/kib_Свердловская область_Екатеринбург_ул. Первомайская, 35.csv')
df = df[df.region.isnull()]
for file in files:
    _, ext = os.path.splitext(file)
    if ext != '.csv':
        continue
    curr_df = pd.read_csv('./parsed_data/' + file)
    df = pd.concat([df, curr_df])

In [7]:
# df.to_csv('parsed_data/kib_parsing_2022_01_30.csv', index=False, encoding='utf-16')

In [5]:
df.to_excel('parsed_data/kib_parsing_2022_01_30.xlsx', index=False)